In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from damage_gradient import solve_problem
from generate_mesh import generate_compact_tension


problem_type = "CT"
W = 8.0
a = 3.0
B = 4.0
e = 0.25
theta = 0*np.pi/180 
coarse_size = 0.5
fine_size = 0.05
x_tip = a + 0.25 * W
y_tip = 0
domain, subdomains, facets = generate_compact_tension(problem_type, W, a, e, theta, coarse_size, fine_size)
geometry_params = (W, a, B, x_tip, y_tip)


# Material parameters
E = 1.8e3
nu = 0.3
KIc = 0.95
Gc = KIc**2 / E * 1e3
l0 = 0.2
model = "AT1"
Nincr = 50
Umax = 0.4
mech_params = (E, nu, Gc, l0, model, Umax, Nincr)

# Alternate minimization parameters
Niter_max = 200
tol = 1e-6

# boundary conditions

def top_pin(x):
    return np.logical_and(np.isclose(x[0], 0.25 * W), np.isclose(x[1], 0.275 * W))


def bot_pin(x):
    return np.logical_and(np.isclose(x[0], 0.25 * W), np.isclose(x[1], -0.275 * W))


def generate_bcs(dirichlet, Uimp):
    dirichlet.add_bc_geometrical(bot_pin, ux=0, uy=0)
    dirichlet.add_bc_geometrical(top_pin, ux=0, uy=Uimp)


prob_params = (problem_type, generate_bcs, Niter_max, tol)
results, u, d, sig = solve_problem(domain, subdomains, facets, geometry_params, prob_params, mech_params)

In [ ]:
Uimp = results[:, 0]
F = results[:, 1]
elastic = results[:, 2]
fracture = results[:, 3]


plt.plot(Uimp, F)
plt.xlabel("Imposed displacement $U$ [mm]")
plt.ylabel("Force [N]")
plt.show()


print(f"Maximum force {max(F):.3f}")